In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c whale-categorization-playground -p /content/
!unzip /content/whale-categorization-playground.zip -d /content/

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: /content/train/7ddecfc2.jpg  
  inflating: /content/train/7de1f190.jpg  
  inflating: /content/train/7de5e1ae.jpg  
  inflating: /content/train/7dea560c.jpg  
  inflating: /content/train/7dea57a7.jpg  
  inflating: /content/train/7df2a97d.jpg  
  inflating: /content/train/7df67375.jpg  
  inflating: /content/train/7df73caa.jpg  
  inflating: /content/train/7df93c7c.jpg  
  inflating: /content/train/7dff8da6.jpg  
  inflating: /content/train/7e058251.jpg  
  inflating: /content/train/7e0a6bfc.jpg  
  inflating: /content/train/7e0b8a41.jpg  
  inflating: /content/train/7e22aca7.jpg  
  inflating: /content/train/7e33bf8a.jpg  
  inflating: /content/train/7e4cd196.jpg  
  inflating: /content/train/7e5496b3.jpg  
  inflating: /content/train/7e54d725.jpg  
  inflating: /content/train/7e55559a.jpg  
  inflating: /content/train/7e5f6d38.jpg  
  inflating: /content/train/7e6c5122.jpg  
  inflating: /content/train/7e

Cargar DataFrame principal

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
df = pd.read_csv('/content/train.csv')
df.head()



,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


Revisar estructura general y nulos

Verificar columnas y tipos

In [ ]:
df.info()
df.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9850 entries, 0 to 9849
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Image   9850 non-null   object
 1   Id      9850 non-null   object
dtypes: object(2)
memory usage: 154.0+ KB


,0
Image,0
Id,0


Eliminar columnas innecesarias

In [ ]:
# Si hubiera alguna columna irrelevante, se eliminaría así:
# df = df.drop(columns=['columna_innecesaria'])


 Revisar duplicados

In [ ]:
dups = df.duplicated().sum()
print(f"Duplicados totales: {dups}")
df = df.drop_duplicates()


Duplicados totales: 0


Contar muestras por especie

In [ ]:
counts = df['Id'].value_counts().reset_index()
counts.columns = ['Id', 'count']
counts['log_count'] = counts['count'].apply(lambda x: np.log1p(x))
counts.head()



,Id,count,log_count
0,new_whale,810,6.698268
1,w_1287fbc,34,3.555348
2,w_98baff9,27,3.332205
3,w_7554f44,26,3.295837
4,w_1eafe46,23,3.178054


Estadísticas descriptivas de frecuencia

In [ ]:
counts['count'].describe()



,count
count,4251.000000
mean,2.317102
std,12.586066
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,810.000000


In [ ]:
print('new_whale' in df['Id'].values)
print('new_whale' in pd.read_csv('sample_submission.csv')['Id'].values)



True
False


Visualizar distribución logarítmica

In [ ]:
import pandas as pd
import numpy as np
import cv2
import skimage
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# Ruta de imágenes
img_path = "train"
# Cargar CSV
df = pd.read_csv('/content/train.csv')
# Obtener paths y etiquetas a partir de df_filtered
file_paths = [f"{img_path}/{img}" for img in df['Image']]
y = df['Id'].values
# Cargar imágenes
imageSize = 100
def get_data(file_paths):
    X = []
    for image_filename in tqdm(file_paths):
        img_file = cv2.imread(image_filename)
        if img_file is not None:
            img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
            img_arr = np.asarray(img_file)
            X.append(img_arr)
    return np.asarray(X)
X = get_data(file_paths)
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Total imágenes cargadas: {len(X)}")
print(f"Train: {len(train_X)}, Val: {len(val_X)}")
print(f"Clases en train: {len(np.unique(train_y))}, en val: {len(np.unique(val_y))}")

100%|██████████| 9850/9850 [12:44<00:00, 12.88it/s]


Total imágenes cargadas: 9850
Train: 7880, Val: 1970
Clases en train: 3769, en val: 1411


Verificar forma de los datos

In [ ]:
print(f"Forma de train_X: {train_X.shape}")
print(f"Forma de val_X: {val_X.shape}")
print(f"Ejemplo de imagen: {train_X[0].shape}, tipo: {type(train_X[0])}")


Forma de train_X: (7880, 100, 100, 3)
Forma de val_X: (1970, 100, 100, 3)
Ejemplo de imagen: (100, 100, 3), tipo: <class 'numpy.ndarray'>
